In [19]:
import subprocess, sys

packages = [
    "langchain-ollama",
    "langchain-core",
    "sentence-transformers",
    "scikit-learn",
    "pandas",
    "openpyxl",
    "tqdm",
]
for pkg in packages:
    subprocess.run([sys.executable, "-m", "pip", "install", pkg, "-q"], check=True)

print("Đã cài đặt tất cả thư viện")

import subprocess, time, os, threading

Đã cài đặt tất cả thư viện


In [20]:
# Install zstd, a dependency required by Ollama's install script
import subprocess
import sys

print(" Đang cài đặt zstd...")
install_zstd_proc = subprocess.run(["apt-get", "update"], capture_output=True, text=True)
if install_zstd_proc.returncode != 0:
    print(f"Lỗi khi cập nhật apt: {install_zstd_proc.stderr}")

install_zstd_proc = subprocess.run(["apt-get", "install", "-y", "zstd"], capture_output=True, text=True)
if install_zstd_proc.returncode != 0:
    print(f"Lỗi khi cài đặt zstd: {install_zstd_proc.stderr}")
else:
    print("zstd đã cài đặt xong.")

def install_and_start_ollama():
    print("Đang cài Ollama...")
    # Using Popen to capture stdout and stderr separately for clarity
    install_proc = subprocess.Popen(
        "curl -fsSL https://ollama.com/install.sh | sh",
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    stdout, stderr = install_proc.communicate()

    if install_proc.returncode != 0:
        print(f"Lỗi cài Ollama (mã lỗi: {install_proc.returncode}).")
        if stdout: print(f"Standard Output:\n{stdout}")
        if stderr: print(f"Standard Error:\n{stderr}")
        return False
    print("Ollama đã cài xong")

    # Add /usr/local/bin to PATH if not already present
    if "/usr/local/bin" not in os.environ["PATH"]:
        os.environ["PATH"] += ":/usr/local/bin"
        print("Đã thêm /usr/local/bin vào PATH.")

    # Verify ollama executable now exists in PATH
    try:
        subprocess.run(["ollama", "--version"], check=True, capture_output=True, text=True)
        print("Ollama executable đã được tìm thấy.")
    except FileNotFoundError:
        print(f"Lỗi: Ollama executable vẫn không tìm thấy sau cài đặt. PATH hiện tại: {os.environ['PATH']}")
        return False
    except subprocess.CalledProcessError as e:
        print(f"Lỗi khi kiểm tra phiên bản Ollama: {e.stderr}")
        return False

    # Khởi động Ollama server trong background
    print(" Khởi động Ollama server...")
    # Use Popen and check=False as it's a background process
    server_proc = subprocess.Popen(
        ["ollama", "serve"], # Should find 'ollama' now via updated PATH
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    time.sleep(5)  # Chờ server khởi động

    # Kiểm tra server
    check_proc = subprocess.run(
        ["ollama", "list"], # Should find 'ollama' now via updated PATH
        capture_output=True, text=True
    )
    if check_proc.returncode == 0:
        print("Ollama server đang chạy")
        return True
    else:
        print(f"Ollama server chưa sẵn sàng, chờ thêm... (mã lỗi: {check_proc.returncode})")
        if check_proc.stdout: print(f"Server check stdout:\n{check_proc.stdout}")
        if check_proc.stderr: print(f"Server check stderr:\n{check_proc.stderr}")
        time.sleep(5) # Wait a bit more
        check_proc = subprocess.run(
            ["ollama", "list"],
            capture_output=True, text=True
        )
        if check_proc.returncode == 0:
            print("Ollama server đang chạy sau khi chờ thêm")
            return True
        else:
            print(f"Ollama server không khởi động được sau khi chờ (mã lỗi: {check_proc.returncode}).")
            if check_proc.stdout: print(f"Final server check stdout:\n{check_proc.stdout}")
            if check_proc.stderr: print(f"Final server check stderr:\n{check_proc.stderr}")
            return False

# ── Chọn model ──────────────────────────────────────────────
# Gợi ý theo VRAM:
#   qwen2.5:7b   → ~6GB VRAM  (T4 free)  — tốt nhất cho tiếng Việt
#   llama3.2:3b  → ~3GB VRAM  (T4 free)  — nhanh hơn
#   qwen2.5:14b  → ~12GB VRAM (Colab Pro) — chất lượng cao nhất

OLLAMA_MODEL = "qwen2.5:7b"   # ← Đổi model tại đây nếu cần

# Check the return value of install_and_start_ollama()
if install_and_start_ollama():
    print(f"\n Đang pull model {OLLAMA_MODEL} (có thể mất vài phút)...")
    pull_result = subprocess.run(
        ["ollama", "pull", OLLAMA_MODEL],
        capture_output=True, text=True
    )
    if pull_result.returncode == 0:
        print(f"Model {OLLAMA_MODEL} sẵn sàng!")
    else:
        print(f"Lỗi pull model: {pull_result.returncode}")
        print("Standard Output:")
        print(pull_result.stdout)
        print("Standard Error:")
        print(pull_result.stderr)
else:
    print("Dừng thực thi do lỗi cài đặt/khởi động Ollama.")

 Đang cài đặt zstd...
zstd đã cài đặt xong.
Đang cài Ollama...
Ollama đã cài xong
Ollama executable đã được tìm thấy.
 Khởi động Ollama server...
Ollama server đang chạy

 Đang pull model qwen2.5:7b (có thể mất vài phút)...
Model qwen2.5:7b sẵn sàng!


In [21]:

from google.colab import files as colab_files
import json

# Upload ragas_dataset.json từ máy lên Colab
print("Hãy upload file ragas_dataset.json:")
uploaded = colab_files.upload()

# Đọc dataset
dataset_filename = list(uploaded.keys())[0]
with open(dataset_filename, encoding="utf-8") as f:
    data = json.load(f)

samples = data.get("samples", data) if isinstance(data, dict) else data
print(f"\nĐã load {len(samples)} samples từ '{dataset_filename}'")
print(f"   Ví dụ câu hỏi đầu: {samples[0]['question'][:60]}...")


LIMIT = None   # ← Đổi thành 10, 20,... nếu muốn test nhanh

# ── Delay giữa các LLM calls (giây) ─────────────────────────
CALL_DELAY = 0.3   # Local model không bị rate limit → delay nhỏ

# ── Output ───────────────────────────────────────────────────
OUTPUT_DIR = "/content/drive/MyDrive/KLTN/ragas_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Model       : {OLLAMA_MODEL}")
print(f"Samples     : {LIMIT or len(samples)} / {len(samples)}")
print(f"Output dir  : {OUTPUT_DIR}")

from langchain_ollama import ChatOllama
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

print(f" Kết nối Ollama model: {OLLAMA_MODEL}")
llm = ChatOllama(model=OLLAMA_MODEL, temperature=0)

# Test nhanh
test_response = llm.invoke("Trả lời chỉ YES hoặc NO: Mặt trời mọc ở phía Đông?").content
print(f"   Test LLM: '{test_response.strip()[:30]}' ✅")

print("\n Đang tải embedding model...")
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
print("Embedding model sẵn sàng")

import re, time

def llm_call(prompt: str) -> str:
    """Gọi LLM local, không cần retry vì không có rate limit."""
    try:
        return llm.invoke(prompt).content.strip()
    except Exception as e:
        print(f"        LLM error: {e}")
        return ""


Hãy upload file ragas_dataset.json:


Saving ragas_dataset.json to ragas_dataset (2).json

Đã load 100 samples từ 'ragas_dataset (2).json'
   Ví dụ câu hỏi đầu: Nếu em từng bị cảnh cáo học vụ ở học kỳ trước nhưng học kỳ n...
Model       : qwen2.5:7b
Samples     : 100 / 100
Output dir  : /content/drive/MyDrive/KLTN/ragas_output
 Kết nối Ollama model: qwen2.5:7b
   Test LLM: 'YES' ✅

 Đang tải embedding model...


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embedding model sẵn sàng


In [15]:
# ── METRIC 1: FAITHFULNESS ──────────────────────────────────
def _extract_claims(answer: str) -> list:
    prompt = f"""Phân tích câu trả lời sau và liệt kê TẤT CẢ các mệnh đề có trong đó.
Mỗi mệnh đề là một thông tin/sự kiện cụ thể, không thể chia nhỏ hơn.

Câu trả lời:
{answer}

Liệt kê các mệnh đề, mỗi mệnh đề bắt đầu bằng "- ":"""
    response = llm_call(prompt)
    claims = [l.lstrip("- ").strip() for l in response.split("\n")
              if l.strip().startswith("-") and len(l.strip()) > 2]
    return claims if claims else [answer]


def _verify_claim(claim: str, context: str) -> bool:
    prompt = f"""Context:
{context[:2500]}

Mệnh đề: {claim}

Mệnh đề này có thể được suy ra từ context không?
Trả lời CHỈ: YES hoặc NO"""
    return "YES" in llm_call(prompt).upper()


def faithfulness(answer: str, contexts: list) -> float:
    if not answer or not contexts:
        return 0.0
    ctx = "\n---\n".join(contexts)
    claims = _extract_claims(answer)
    supported = sum(_verify_claim(c, ctx) for c in claims)
    score = supported / len(claims)
    print(f"      Faithfulness: {supported}/{len(claims)} → {score:.2f}")
    return score


# ── METRIC 2: ANSWER RELEVANCY ──────────────────────────────
def _gen_questions(answer: str, n: int = 3) -> list:
    prompt = f"""Đọc câu trả lời sau và sinh {n} câu hỏi mà câu trả lời này có thể trả lời được.

Câu trả lời:
{answer}

Sinh {n} câu hỏi, mỗi câu bắt đầu bằng "- ":"""
    response = llm_call(prompt)
    return [l.lstrip("- ").strip() for l in response.split("\n")
            if l.strip().startswith("-") and len(l.strip()) > 3][:n]


def answer_relevancy(question: str, answer: str) -> float:
    if not answer or not question:
        return 0.0
    gens = _gen_questions(answer, n=3)
    if not gens:
        emb = embed_model.encode([question, answer])
        score = float(cosine_similarity([emb[0]], [emb[1]])[0][0])
        print(f"      Answer Relevancy (fallback embed): {score:.2f}")
        return max(0.0, min(1.0, score))
    q_emb = embed_model.encode([question])
    g_emb = embed_model.encode(gens)
    score = float(cosine_similarity(q_emb, g_emb)[0].mean())
    score = max(0.0, min(1.0, score))
    print(f"      Answer Relevancy: {score:.2f}")
    return score


# ── METRIC 3: CONTEXT PRECISION ─────────────────────────────
def _is_relevant(question: str, chunk: str, ground_truth: str) -> bool:
    prompt = f"""Câu hỏi: {question}
Đáp án mong đợi: {ground_truth}

Đoạn context:
{chunk[:1200]}

Đoạn context này có chứa thông tin hữu ích để trả lời câu hỏi không?
Trả lời CHỈ: YES hoặc NO"""
    return "YES" in llm_call(prompt).upper()


def context_precision(question: str, contexts: list, ground_truth: str) -> float:
    if not contexts:
        return 0.0
    relevances = [_is_relevant(question, c, ground_truth) for c in contexts]
    num, den, running = 0.0, 0.0, 0
    for k, r in enumerate(relevances, 1):
        if r:
            running += 1
            num += running / k
            den += 1
    score = (num / den) if den > 0 else 0.0
    print(f"      Context Precision: {sum(relevances)}/{len(contexts)} → {score:.2f}")
    return score


# ── METRIC 4: CONTEXT RECALL ────────────────────────────────
def _extract_statements(ground_truth: str) -> list:
    prompt = f"""Phân tích đáp án chuẩn sau và liệt kê TẤT CẢ các thông tin/sự kiện cụ thể.

Đáp án chuẩn:
{ground_truth}

Liệt kê các thông tin, mỗi thông tin bắt đầu bằng "- ":"""
    response = llm_call(prompt)
    stmts = [l.lstrip("- ").strip() for l in response.split("\n")
             if l.strip().startswith("-") and len(l.strip()) > 2]
    return stmts if stmts else [ground_truth]


def _stmt_in_context(statement: str, context: str) -> bool:
    prompt = f"""Context:
{context[:2500]}

Statement: {statement}

Statement này có thể được suy ra từ context không?
Trả lời CHỈ: YES hoặc NO"""
    return "YES" in llm_call(prompt).upper()


def context_recall(contexts: list, ground_truth: str) -> float:
    if not contexts or not ground_truth:
        return 0.0
    ctx = "\n---\n".join(contexts)
    stmts = _extract_statements(ground_truth)
    supported = sum(_stmt_in_context(s, ctx) for s in stmts)
    score = supported / len(stmts)
    print(f"      Context Recall: {supported}/{len(stmts)} → {score:.2f}")
    return score

print("Các hàm RAGAS đã sẵn sàng")


Các hàm RAGAS đã sẵn sàng


In [22]:
import pandas as pd
from datetime import datetime

# Slice samples
eval_samples = samples[:LIMIT] if LIMIT else samples
total = len(eval_samples)

print("=" * 65)
print(f" BẮT ĐẦU ĐÁNH GIÁ RAGAS — {total} SAMPLES")
print(f"   Model judge : {OLLAMA_MODEL}")
print(f"   Embed model : paraphrase-multilingual-MiniLM-L12-v2")
print("=" * 65)

results = []
t_start = time.time()

for i, sample in enumerate(eval_samples, 1):
    question     = sample["question"]
    answer       = sample["answer"]
    contexts     = sample["contexts"]
    ground_truth = sample["ground_truth"]
    qid          = sample.get("question_id", str(i))

    print(f"\n[{i:3}/{total}] {question[:60]}{'...' if len(question)>60 else ''}")

    if answer.startswith("ERROR:") or not contexts:
        print("        Bỏ qua (lỗi hoặc không có context)")
        results.append({
            "question_id": qid, "question": question,
            "faithfulness": None, "answer_relevancy": None,
            "context_precision": None, "context_recall": None,
            "ragas_score": None, "error": "no_context_or_error"
        })
        continue

    t0 = time.time()
    f  = faithfulness(answer, contexts)
    r  = answer_relevancy(question, answer)
    p  = context_precision(question, contexts, ground_truth)
    rc = context_recall(contexts, ground_truth)

    ragas_avg = (f + r + p + rc) / 4
    elapsed   = time.time() - t0

    print(f"       RAGAS: {ragas_avg:.3f}  [{elapsed:.0f}s]")
    results.append({
        "question_id":       qid,
        "question":          question,
        "ground_truth":      ground_truth,
        "answer":            answer,
        "num_contexts":      len(contexts),
        "faithfulness":      round(f,  4),
        "answer_relevancy":  round(r,  4),
        "context_precision": round(p,  4),
        "context_recall":    round(rc, 4),
        "ragas_score":       round(ragas_avg, 4),
        "error":             None,
    })

    time.sleep(CALL_DELAY)

total_time = time.time() - t_start

valid   = [r for r in results if r["ragas_score"] is not None]

def avg(key):
    # Exclude None values and 0.0 values from the overall average calculation
    vals = [r[key] for r in valid if r[key] is not None and r[key] > 0]
    return sum(vals) / len(vals) if vals else 0.0

benchmarks = {
    "faithfulness":      0.85,
    "answer_relevancy":  0.85,
    "context_precision": 0.75,
    "context_recall":    0.80,
}
metrics = {k: avg(k) for k in benchmarks}
metrics["ragas_score"] = avg("ragas_score")

 BẮT ĐẦU ĐÁNH GIÁ RAGAS — 100 SAMPLES
   Model judge : qwen2.5:7b
   Embed model : paraphrase-multilingual-MiniLM-L12-v2

[  1/100] Nếu em từng bị cảnh cáo học vụ ở học kỳ trước nhưng học kỳ n...
      Faithfulness: 1/4 → 0.25
      Answer Relevancy: 0.75
      Context Precision: 2/4 → 0.42
      Context Recall: 1/7 → 0.14
       RAGAS: 0.391  [25s]

[  2/100] Sinh viên chưa đóng đủ học phí có được xét học bổng không, v...
      Faithfulness: 8/9 → 0.89
      Answer Relevancy: 0.92
      Context Precision: 0/4 → 0.00
      Context Recall: 4/8 → 0.50
       RAGAS: 0.577  [25s]

[  3/100] Cảnh cáo học vụ khác gì với cảnh cáo kỷ luật, và mỗi loại ản...
      Faithfulness: 3/8 → 0.38
      Answer Relevancy: 0.82
      Context Precision: 2/4 → 1.00
      Context Recall: 0/7 → 0.00
       RAGAS: 0.549  [27s]

[  4/100] Không tham gia hoạt động Đoàn – Hội thì điểm rèn luyện thấp ...
      Faithfulness: 5/5 → 1.00
      Answer Relevancy: 0.81
      Context Precision: 2/2 → 1.00
      Context R

In [24]:

print("\n" + "=" * 65)
print("KẾT QUẢ RAGAS")
print("=" * 65)
print(f"{'Metric':<22} {'Score':>7}")
print("─" * 30)

for key in ["faithfulness", "answer_relevancy", "context_precision", "context_recall"]:
    score = metrics[key]
    print(f"{key.replace('_',' ').title():<22} {score:>7.4f}")

# Calculate RAGAS Average as the average of the four metrics
ragas_overall_avg = (metrics["faithfulness"] + metrics["answer_relevancy"] + metrics["context_precision"] + metrics["context_recall"]) / 4

print("─" * 30)
print(f"{'RAGAS Average':<22} {ragas_overall_avg:>7.4f}")
print(f"\n  Samples  : {len(valid)}/{total} hợp lệ")
print(f"  Thời gian: {total_time/60:.1f} phút")
print(f"  Model    : {OLLAMA_MODEL}")
print("=" * 65)

# Xuất Excel
df        = pd.DataFrame(results)
ts        = datetime.now().strftime("%Y%m%d_%H%M%S")
xlsx_path = f"{OUTPUT_DIR}/ragas_report_{ts}.xlsx"
df.to_excel(xlsx_path, index=False)
print(f"\nBáo cáo Excel: {xlsx_path}")

# Xuất JSON summary
import json
summary = {
    "evaluated_at":  datetime.now().isoformat(),
    "model_judge":   OLLAMA_MODEL,
    "total_samples": total,
    "valid_samples": len(valid),
    "metrics":       {k: round(v, 4) for k, v in metrics.items()},
    "total_time_min": round(total_time / 60, 1),
}
# Update the ragas_score in the summary with the newly calculated overall average
summary["metrics"]["ragas_score"] = round(ragas_overall_avg, 4)

json_path = f"{OUTPUT_DIR}/ragas_summary_{ts}.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)
print(f"Summary JSON : {json_path}")


from google.colab import files as colab_files

print("Đang tải file về máy...")
colab_files.download(xlsx_path)
colab_files.download(json_path)
print("Đã download!")


KẾT QUẢ RAGAS
Metric                   Score
──────────────────────────────
Faithfulness            0.5021
Answer Relevancy        0.7642
Context Precision       0.9430
Context Recall          0.3414
──────────────────────────────
RAGAS Average           0.6377

  Samples  : 100/100 hợp lệ
  Thời gian: 46.9 phút
  Model    : qwen2.5:7b

Báo cáo Excel: /content/drive/MyDrive/KLTN/ragas_output/ragas_report_20260221_112759.xlsx
Summary JSON : /content/drive/MyDrive/KLTN/ragas_output/ragas_summary_20260221_112759.json
Đang tải file về máy...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Đã download!
